In [1]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import seaborn as sns
import threading
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor
from pymongo import MongoClient, errors

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 999)

In [2]:
def concat_regular_parquet_files(folder_path, output_path=None, recursive=False):
    all_files = []
    
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".parquet") and not file.endswith("_noos_adjusted.parquet"):
                all_files.append(os.path.join(root, file))
        if not recursive:
            break

    if not all_files:
        raise FileNotFoundError("No .parquet files found in the given folder.")

    df_list = [pd.read_parquet(fp) for fp in all_files]
    combined_df = pd.concat(df_list, ignore_index=True)

    if output_path:
        combined_df.to_parquet(output_path, index=False)
        print(f"✅ Saved concatenated file to: {output_path}")

    return combined_df

def concat_noos_parquet_files(folder_path, output_path=None, recursive=False):
    all_files = []
    
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".parquet") and file.endswith("_noos_adjusted.parquet"):
                all_files.append(os.path.join(root, file))
        if not recursive:
            break

    if not all_files:
        raise FileNotFoundError("No .parquet files found in the given folder.")

    df_list = [pd.read_parquet(fp) for fp in all_files]
    combined_df = pd.concat(df_list, ignore_index=True)

    if output_path:
        combined_df.to_parquet(output_path, index=False)
        print(f"✅ Saved concatenated file to: {output_path}")

    return combined_df

In [3]:
regular_forecast = concat_regular_parquet_files("/Users/trentino/Work/OFM/droplet/2026 Forecast/forecasted_data")

In [4]:
sales_data = pd.read_parquet("/Users/trentino/Work/OFM/droplet/2026 Forecast/data/clean_sales_data.parquet")

In [5]:
internal_brands = ['Campbell','Donkervoort','Dutch Dandies','Recall','The BLUEPRINT Premium','Nobel','Runway PARTY','J.C. RAGS']
internal_brands_list = [618, 427, 301, 438, 228, 1000, 804, 876]

In [6]:
regular_sales_data = sales_data[sales_data['season'].isin(["Summer", "Winter"])].reset_index(drop=True)

In [7]:
all_season_others = ['Rugby', 'Slippers', 'Parka', 'Spencer', 'Dress Shirt extra long sleeve', 'Grandad', 'Casual shirt short sleeve', 'Sweatpants', 'T-shirt LS', 'Dress Shirt long sleeve', 'Gloves', 
                     'T-shirt long sleeve', 'Polo short sleeve', 'Pocket square', 'Bow', 'T-shirt short sleeve', 'Pullover half zip', 'Boat Shoes', 'Knitwear', 'Dress belt', 'Boots', 'Casual belt', 
                     'Leather jacket', 'Boxershort', 'Bag', 'Shirt', 'Suspenders', 'Waistcoat', 'Others', 'Dress Shirt SS', 'Loafers', 'Trendy shirt long sleeve', 'Polo long sleeve', 'Shawl', 
                     'Casual shirt long sleeve', 'Blazer', 'Bodywarmer', 'Watch']

regular_articles = ['Trendy shirt LS', 'Casual shirt LS', 'Chino', 'Dress Shirt LS', 'Cardigan', 'Jeans', 'Sweatshirt', 'Jacket', '5-pocket', 'Dress Shoe', 'Overshirt', 'Polo LS', 'Hoodie', 'Trousers', 
                    'V-neck sweater', 'Casual Shoe', 'Suit', 'Mix & match jacket', 'Headwear', 'Socks', "Others"]

season = ["Winter", "Summer"]

In [8]:
regular_sales_data.loc[(regular_sales_data["articleGroupDescription"].isin(all_season_others)) & (regular_sales_data['season'].isin(season)), "articleGroupDescription"] = "Others"

regular_sales_data = regular_sales_data[regular_sales_data["articleGroupDescription"].isin(regular_articles)].reset_index(drop=True)
uncombined_sales_data = regular_sales_data.copy()
regular_sales_data = regular_sales_data.groupby(["salesDate", "articleGroupDescription", "brandDescription", "Inhouse_Brand"]).agg({"quantity":"sum"}).reset_index()

In [9]:
regular_forecast.loc[(regular_forecast['articleGroupDescription'].isin(regular_articles)) & (regular_forecast['Season'].isna()), "Season"] = "All Season"

In [10]:
regular_forecast[regular_forecast['Season'] == "All Season"]

,salesDate,articleGroupDescription,Approach,SalesForecast,Season
1,2026-01-01 to 2027-04-01,Mix & match jacket,Adj: Historical,5478.0,All Season
3,2026-01-01 to 2027-04-01,Trousers,Adj: Historical,4708.0,All Season
4,2026-01-01 to 2027-04-01,Jeans,Adj: Historical,12044.0,All Season
6,2026-01-01 to 2027-04-01,Socks,Raw Model,3636.0,All Season
7,2026-01-01 to 2027-04-01,Chino,Adj: Historical,35010.0,All Season
10,2026-01-01 to 2027-04-01,Others,Adj: Historical,22142.0,All Season
14,2026-01-01 to 2027-04-01,Polo LS,Adj: Historical,7170.0,All Season
17,2026-01-01 to 2027-04-01,5-pocket,Adj: Pace,2445.0,All Season
18,2026-01-01 to 2027-04-01,Casual Shoe,Adj: Pace,3305.0,All Season
20,2026-01-01 to 2027-04-01,Headwear,Adj: Historical,5229.0,All Season


In [11]:
regular_forecast = regular_forecast[regular_forecast['Season'] == "All Season"]

In [12]:
inhouse_brands = ['Campbell',
 'Donkervoort',
 'Dutch Dandies',
 'Recall',
 'The BLUEPRINT Premium',
 'Nobel',
 'Runway PARTY',
 'J.C. RAGS']

regular_sales_data.loc[~(regular_sales_data["brandDescription"].isin(inhouse_brands)), "brandDescription"] = "Others"

In [13]:
def allocate_forecast_to_brands(sales_df, forecast_df, inhouse_brands, historical_weight=0.5):
    sales_df = sales_df.copy()
    sales_df["salesDate"] = pd.to_datetime(sales_df["salesDate"])

    # Define date ranges
    hist_mask = (sales_df["salesDate"].dt.month >= 1) & (sales_df["salesDate"].dt.month <= 12) & (sales_df["salesDate"].dt.year.isin([2023, 2024]))
    recent_mask = (sales_df["salesDate"] >= "2025-01-01") & (sales_df["salesDate"] <= "2025-12-31")

    def compute_brand_share(df, label):
        group = df.groupby(["articleGroupDescription", "brandDescription"])["quantity"].sum().reset_index()
        total = group.groupby("articleGroupDescription")["quantity"].transform("sum")
        group["share"] = group["quantity"] / total
        group.rename(columns={"share": f"{label}_share"}, inplace=True)
        return group[["articleGroupDescription", "brandDescription", f"{label}_share"]]

    hist_share = compute_brand_share(sales_df[hist_mask], "hist")
    recent_share = compute_brand_share(sales_df[recent_mask], "recent")

    # Merge and blend
    merged = pd.merge(hist_share, recent_share, on=["articleGroupDescription", "brandDescription"], how="outer").fillna(0)
    merged["blended_share"] = historical_weight * merged["hist_share"] + (1 - historical_weight) * merged["recent_share"]

    # Merge forecasted article sales
    forecast_df = forecast_df.copy()
    merged_forecast = pd.merge(forecast_df, merged, on="articleGroupDescription", how="left")

    # Calculate brand-level forecast
    merged_forecast["BrandForecast"] = merged_forecast["SalesForecast"] * merged_forecast["blended_share"]
    merged_forecast["BrandForecast"] = round(merged_forecast["BrandForecast"], 0)
    
    # Tag internal/external brands
    merged_forecast["Inhouse_Brand"] = merged_forecast["brandDescription"].isin(inhouse_brands)

    return merged_forecast[[
        "salesDate", "articleGroupDescription", "brandDescription", "Inhouse_Brand", 
        "Approach", "SalesForecast", "blended_share", "BrandForecast"
    ]]

def historical_brand_breakdown(sales_data, start_year=2023, end_year=2024, inhouse_brands=None):
    if inhouse_brands is None:
        inhouse_brands = []

    # Filter for historical Mar–Sept sales (exclude 2025)
    hist_mask = (
        (sales_data["salesDate"].dt.year >= start_year) &
        (sales_data["salesDate"].dt.year <= end_year) &
        (sales_data["salesDate"].dt.month >= 1) &
        (sales_data["salesDate"].dt.month <= 12)
    )
    hist_df = sales_data.loc[hist_mask].copy()

    # Group by article + brand and sum quantities
    group = hist_df.groupby(["articleGroupDescription", "brandDescription"])["quantity"].sum().reset_index()

    # Compute total sales per article to get brand share
    total = group.groupby("articleGroupDescription")["quantity"].transform("sum")
    group["brand_share"] = group["quantity"] / total

    # Also calculate total article-level sales
    article_totals = group.groupby("articleGroupDescription")["quantity"].sum().reset_index()
    article_totals = article_totals.rename(columns={"quantity": "article_total_sales"})

    # Merge article totals back to group
    merged = pd.merge(group, article_totals, on="articleGroupDescription")

    # Estimate brand-level "forecast-like" numbers from historical totals
    merged["brand_quantity_estimate"] = merged["brand_share"] * merged["article_total_sales"]

    # Flag in-house brands
    merged["Inhouse_Brand"] = merged["brandDescription"].isin(inhouse_brands)

    return merged

In [14]:
historical_brand_summary = historical_brand_breakdown(regular_sales_data, start_year=2023, end_year=2025, inhouse_brands=None)

grouped = historical_brand_summary.groupby("Inhouse_Brand").agg({"brand_quantity_estimate": "sum"}).reset_index()
total_quantity = grouped["brand_quantity_estimate"].sum()
grouped["contribution_%"] = grouped["brand_quantity_estimate"] /  total_quantity * 100
grouped["contribution_%"] = grouped["contribution_%"].round(2)
grouped

,Inhouse_Brand,brand_quantity_estimate,contribution_%
0,False,660702.0,100.0


In [15]:
# brands = list(sales_data['brandDescription'].dropna().unique())

In [16]:
bifurcated_sales = allocate_forecast_to_brands(regular_sales_data, regular_forecast, inhouse_brands)
grouped = bifurcated_sales.groupby("Inhouse_Brand").agg({
    "BrandForecast": "sum"
}).reset_index()
total_quantity = grouped["BrandForecast"].sum()
grouped["contribution_%"] = grouped["BrandForecast"] / total_quantity * 100
grouped["contribution_%"] = grouped["contribution_%"].round(2)
grouped

,Inhouse_Brand,BrandForecast,contribution_%
0,False,149953.0,57.5
1,True,110824.0,42.5


In [17]:
sales_forecasted = bifurcated_sales[["salesDate", "articleGroupDescription", "brandDescription", "Inhouse_Brand", "BrandForecast"]]

sales_forecasted = sales_forecasted.rename(columns = {"BrandForecast":"quantity"})

sales_forecasted["category"] = "Forecasted Sales"

sales_forecasted["Inhouse_Brand"] = sales_forecasted["Inhouse_Brand"].replace({True:"Internal", False:"External"})

sales_forecasted["salesDate"] = pd.to_datetime(datetime(2026, 12, 31))

In [18]:
sales_forecasted.head(1)

,salesDate,articleGroupDescription,brandDescription,Inhouse_Brand,quantity,category
0,2026-12-31,Mix & match jacket,Campbell,Internal,178.0,Forecasted Sales


In [19]:
uncombined_sales_data.loc[(uncombined_sales_data["Discount%"] == 0), "IsDiscount"] = True
uncombined_sales_data["IsDiscount"] = uncombined_sales_data["IsDiscount"].fillna(False)

# Step 1: Calculate sales proportions (discount vs non-discount)
grouped = uncombined_sales_data[(uncombined_sales_data["salesDate"] >= "2022-01-01")].groupby(
    ["articleGroupDescription", "brandDescription", "IsDiscount"]
).agg({"quantity": "sum"}).reset_index()

# Total sales per article-brand
totals = grouped.groupby(
    ["articleGroupDescription", "brandDescription"]
)["quantity"].sum().reset_index().rename(columns={"quantity": "total_quantity"})

# Merge to compute proportion
merged = pd.merge(grouped, totals, on=["articleGroupDescription", "brandDescription"])
merged["sales_proportion"] = merged["quantity"] / merged["total_quantity"]

# Pivot into separate columns for True and False
proportions_pivot = merged.pivot(
    index=["articleGroupDescription", "brandDescription"],
    columns="IsDiscount",
    values="sales_proportion"
).reset_index()

# Handle missing columns by filling with 0, then rename
proportions_pivot = proportions_pivot.rename(columns={
    True: "discount_prop",
    False: "non_discount_prop"
})

# Ensure both columns exist
if "discount_prop" not in proportions_pivot.columns:
    proportions_pivot["discount_prop"] = 0.0
if "non_discount_prop" not in proportions_pivot.columns:
    proportions_pivot["non_discount_prop"] = 0.0

# Step 2: Pricing - max price (non-discount), avg price (discounted but less than max)
non_discount_price_df = uncombined_sales_data[(uncombined_sales_data["IsDiscount"] == False) & (uncombined_sales_data["salesDate"] >= "2024-01-01")] \
    .groupby(["articleGroupDescription", "brandDescription"])["retailPrice"].mean().reset_index() \
    .rename(columns={"retailPrice": "non_discount_price"})

discount_df = pd.merge(
    uncombined_sales_data[uncombined_sales_data["IsDiscount"] == True],
    non_discount_price_df,
    on=["articleGroupDescription", "brandDescription"],
    how="inner"
)

discount_df = discount_df[discount_df["retailPrice"] < discount_df["non_discount_price"]]

discount_price_df = uncombined_sales_data[(uncombined_sales_data["salesDate"] >= "2024-01-01")].groupby(["articleGroupDescription", "brandDescription"]) \
    .agg({"retailPrice": "mean"}).reset_index() \
    .rename(columns={"retailPrice": "discount_price"})

# Step 3: Combine price info
price_data_cleaned = pd.merge(non_discount_price_df, discount_price_df,
                              on=["articleGroupDescription", "brandDescription"], how="left")

# Step 4: Combine proportion + pricing
pricing_data = pd.merge(proportions_pivot, price_data_cleaned,
                        on=["articleGroupDescription", "brandDescription"], how="left")

# Step 5: Merge with forecast
sales_forecasted = pd.merge(sales_forecasted, pricing_data,
                            on=["articleGroupDescription", "brandDescription"], how="left")

# Ensure fill for proportions (if any are missing)
sales_forecasted["discount_prop"] = sales_forecasted["discount_prop"].fillna(0)
sales_forecasted["non_discount_prop"] = sales_forecasted["non_discount_prop"].fillna(1)

# Step 6: Project sales and revenue
sales_forecasted["discount_sales"] = sales_forecasted["quantity"] * sales_forecasted["discount_prop"]
sales_forecasted["non_discount_sales"] = sales_forecasted["quantity"] * sales_forecasted["non_discount_prop"]

sales_forecasted["discount_revenue"] = sales_forecasted["discount_sales"] * sales_forecasted["discount_price"]
sales_forecasted["non_discount_revenue"] = sales_forecasted["non_discount_sales"] * sales_forecasted["non_discount_price"]

sales_forecasted["retailPrice"] = (
    sales_forecasted["discount_revenue"].fillna(0) + sales_forecasted["non_discount_revenue"].fillna(0)
).round(2)

# Final output
sales_forecasted = sales_forecasted[[
"salesDate", "articleGroupDescription", "brandDescription", "Inhouse_Brand", "quantity", "category", "retailPrice"
]]

print(sales_forecasted["retailPrice"].sum())

sales_forecasted.head()

6291773.359999999


,salesDate,articleGroupDescription,brandDescription,Inhouse_Brand,quantity,category,retailPrice
0,2026-12-31,Mix & match jacket,Campbell,Internal,178.0,Forecasted Sales,16601.86
1,2026-12-31,Mix & match jacket,Dutch Dandies,Internal,1070.0,Forecasted Sales,170173.30
2,2026-12-31,Mix & match jacket,Others,External,2101.0,Forecasted Sales,0.00
3,2026-12-31,Mix & match jacket,Recall,Internal,500.0,Forecasted Sales,33907.93
4,2026-12-31,Mix & match jacket,Runway PARTY,Internal,1102.0,Forecasted Sales,194937.33


In [20]:
uncombined_sales_data[uncombined_sales_data['articleGroupDescription'].isin(regular_articles)].groupby(["year","articleGroupDescription"]).agg({"quantity":"sum", "retailPrice":"sum"})

quantity  retailPrice
year articleGroupDescription                       
2019 5-pocket                  10356.0   1057470.50
     Cardigan                   6937.0    745745.75
     Casual Shoe                3539.0    509965.38
     Casual shirt LS           29087.0   2113290.13
     Chino                     12372.0   1658588.77
     Dress Shirt LS            22300.0   1797098.88
     Dress Shoe                10086.0   1262965.20
     Headwear                    368.0     16848.08
     Hoodie                      660.0     62704.01
     Jacket                    11228.0   2397939.23
     Jeans                     13548.0   1862358.77
     Mix & match jacket          440.0    152813.14
     Others                    10454.0    883636.80
     Polo LS                    2393.0    224766.81
     Socks                      5040.0     50447.82
     Suit                       8048.0   2670772.90
     Sweatshirt                 7540.0    638255.55
     Trendy shirt LS           45008.0   1807514.14
     Trousers                   2323.0    345972.05
     V-neck sweater             7448.0    467107.53
2020 5-pocket                   8261.0    780299.19
     Cardigan                   7811.0    860504.67
     Casual Shoe                3515.0    420045.68
     Casual shirt LS           23346.0   1563968.96
     Chino                     12638.0   1466988.75
     Dress Shirt LS            19601.0   1418075.34
     Dress Shoe                 9836.0   1175822.45
     Headwear                    484.0     22072.32
     Hoodie                     1249.0    130315.46
     Jacket                     7961.0   1464953.14
     Jeans                     12166.0   1556410.08
     Mix & match jacket          861.0    244403.34
     Others                     9156.0    734467.62
     Overshirt                   148.0     13184.59
     Polo LS                    2666.0    210333.58
     Socks                      5582.0     43572.08
     Suit                       3230.0   1052480.75
     Sweatshirt                 9285.0    711851.81
     Trendy shirt LS           33996.0   1351992.23
     Trousers                   2444.0    293322.04
     V-neck sweater             3501.0    239666.65
2021 5-pocket                   6827.0    818834.65
     Cardigan                  10429.0   1374849.43
     Casual Shoe                2275.0    292417.82
     Casual shirt LS           21887.0   1559384.12
     Chino                     13166.0   1777806.34
     Dress Shirt LS            15759.0   1117704.76
     Dress Shoe                 5537.0    724810.48
     Headwear                   1173.0     57684.95
     Hoodie                     5184.0    539877.74
     Jacket                     7373.0   1617163.13
     Jeans                      8867.0   1347804.09
     Mix & match jacket          979.0    310416.96
     Others                    12250.0   1441494.04
     Overshirt                  1199.0    185336.40
     Polo LS                    4096.0    401952.22
     Socks                       685.0     10371.04
     Suit                       1666.0    771794.79
     Sweatshirt                11045.0    946903.47
     Trendy shirt LS           28634.0   1120932.37
     Trousers                   3721.0    422708.18
     V-neck sweater             4909.0    285391.92
2022 5-pocket                   5865.0    655517.75
     Cardigan                  15456.0   1816708.01
     Casual Shoe                1788.0    302455.95
     Casual shirt LS           24408.0   1937929.92
     Chino                     13899.0   1977065.97
     Dress Shirt LS            14354.0   1336353.06
     Dress Shoe                 5716.0    844445.34
     Headwear                   2979.0    126987.04
     Hoodie                     9349.0    894885.60
     Jacket                     6999.0   1697522.64
     Jeans                     13840.0   2087672.19
     Mix & match jacket         1874.0    514540.84
     Others                    18039.0   2400565.80
     Overshirt            

In [21]:
sales_forecasted.groupby("articleGroupDescription").agg({"quantity":"sum", "retailPrice":"sum"})

,quantity,retailPrice
articleGroupDescription,,
5-pocket,2445.0,3209.43
Cardigan,20121.0,534646.61
Casual Shoe,3305.0,39070.31
Casual shirt LS,40404.0,745405.16
Chino,35010.0,171058.22
Dress Shirt LS,27934.0,324049.33
Dress Shoe,4959.0,150123.93
Headwear,5229.0,15112.49
Jacket,9369.0,1016427.01


In [22]:
external_others_articles = list(sales_forecasted[(sales_forecasted['retailPrice'] == 0.0)]['articleGroupDescription'].unique())

temp_data = sales_data[(sales_data['articleGroupDescription'].isin(external_others_articles)) &
                       (sales_data['Inhouse_Brand'] == False)&
                       (sales_data['season'].isin(['Summer', "Winter"]))].groupby("articleGroupDescription").agg({'quantity':"sum","retailPrice":"sum"}).reset_index()

temp_data['avg_cost'] = round(temp_data['retailPrice'] / temp_data['quantity'], 1)

sales_forecasted = pd.merge(sales_forecasted, temp_data[['articleGroupDescription', 'avg_cost']], on = 'articleGroupDescription', how = 'left')

sales_forecasted.loc[(sales_forecasted['retailPrice'] == 0.0), 'retailPrice'] = round(sales_forecasted['quantity'] * sales_forecasted['avg_cost'], 2)

sales_forecasted = sales_forecasted.drop(columns = 'avg_cost')

In [23]:
sales_forecasted['quantity'].sum(), sales_forecasted['retailPrice'].sum()

(np.float64(260777.0), np.float64(23466298.96))

In [24]:
uncombined_sales_data["year"] = uncombined_sales_data["salesDate"].dt.year
uncombined_sales_data["month"] = uncombined_sales_data["salesDate"].dt.month

In [25]:
uncombined_sales_data[(uncombined_sales_data["month"].isin([1,2,3,4,5,6,7,8,9,10,11,12]))].groupby("year").agg({"retailPrice":"sum", "quantity":'sum'}).reset_index()

,year,retailPrice,quantity
0,2019,20726261.44,209175.0
1,2020,15754730.73,177737.0
2,2021,17125638.90,167661.0
3,2022,20700773.00,187075.0
4,2023,23097517.67,221760.0
5,2024,24777440.18,210259.0
6,2025,25626442.06,228685.0


In [34]:
sales_forecasted

,salesDate,articleGroupDescription,brandDescription,Inhouse_Brand,quantity,category,retailPrice
0,2026-12-31,Mix & match jacket,Campbell,Internal,178.0,Forecasted Sales,16601.86
1,2026-12-31,Mix & match jacket,Dutch Dandies,Internal,1070.0,Forecasted Sales,170173.30
2,2026-12-31,Mix & match jacket,Others,External,2101.0,Forecasted Sales,562437.70
3,2026-12-31,Mix & match jacket,Recall,Internal,500.0,Forecasted Sales,33907.93
4,2026-12-31,Mix & match jacket,Runway PARTY,Internal,1102.0,Forecasted Sales,194937.33
5,2026-12-31,Mix & match jacket,The BLUEPRINT Premium,Internal,528.0,Forecasted Sales,68357.21
6,2026-12-31,Trousers,Campbell,Internal,145.0,Forecasted Sales,7452.77
7,2026-12-31,Trousers,Dutch Dandies,Internal,586.0,Forecasted Sales,36159.33
8,2026-12-31,Trousers,J.C. RAGS,Internal,297.0,Forecasted Sales,15204.49
9,2026-12-31,Trousers,Others,External,3519.0,Forecasted Sales,433892.70


In [35]:
sales_forecasted.loc[sales_forecasted["articleGroupDescription"].isin(regular_articles), "season"] = "Year Round"

In [37]:
sales_forecasted.to_csv("/Users/trentino/Work/OFM/droplet/2026 Forecast/data/sales_forecast_year_round_regular_2026_season.csv", index=False)

In [29]:
# uncombined_sales_data.loc[uncombined_sales_data["articleGroupDescription"].isin(regular_articles), "season"] = "Summer"

In [30]:
combined_summ = uncombined_sales_data.groupby(['year', "Inhouse_Brand", 'season', 'articleGroupDescription'])

In [40]:
combined_summ[(combined_summ['year'] == 2025) & (combined_summ['articleGroupDescription'] == "Others")]

,year,Inhouse_Brand,season,articleGroupDescription,quantity,retailPrice
454,2025,False,Summer,Others,19705.0,1911842.66
474,2025,False,Winter,Others,24725.0,2981150.42
493,2025,True,Summer,Others,12354.0,632132.70
513,2025,True,Winter,Others,21640.0,1177762.62


In [41]:
combined_summ[(combined_summ['year'] == 2024) & (combined_summ['articleGroupDescription'] == "Others")]

,year,Inhouse_Brand,season,articleGroupDescription,quantity,retailPrice
373,2024,False,Summer,Others,4054.0,705726.42
393,2024,False,Winter,Others,7313.0,1369087.79
413,2024,True,Summer,Others,2258.0,171537.57
433,2024,True,Winter,Others,2916.0,211609.52


In [55]:
combined_summ[(combined_summ['year'] == 2024) & (combined_summ['Inhouse_Brand'] == True)].groupby(["articleGroupDescription"]).agg({"quantity":"sum", "retailPrice":"sum"}).reset_index()

,articleGroupDescription,quantity,retailPrice
0,5-pocket,85.0,6965.64
1,Cardigan,7879.0,570817.88
2,Casual Shoe,753.0,66624.70
3,Casual shirt LS,16220.0,744773.92
4,Chino,2713.0,266553.47
5,Dress Shirt LS,1992.0,236083.28
6,Dress Shoe,1496.0,143210.16
7,Headwear,611.0,10689.21
8,Hoodie,408.0,25676.17
9,Jacket,5406.0,1368963.21


383147.08999999997